This notebook is needed because in version 12, I reran step6 creating transit netwok to add missing Amtrak segments, and thus rail nodes and links IDs got shifted, we need to update IDs in:

1. project cards
2. tap files

and this notebook updates the node IDs in tap files, it is needed to run just ONCE

In [2]:
import pandas as pd
import os

# Remote I/O

In [3]:
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"
tap_file_dir = os.path.join(lasso_dir, 'mtc_data', 'tap')

root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
node_id_crosswalk_file = os.path.join(root_dir, 'data', 'processed', 'version_12', 'node_id_crosswalk.csv')

out_dir = tap_file_dir

# Read files

In [30]:
tap_nodes_df = pd.read_pickle(os.path.join(tap_file_dir, 'tap_node.pickle'))

In [4]:
tap_links_df = pd.read_pickle(os.path.join(tap_file_dir, 'tap_link.pickle'))

In [23]:
tap_shapes_df = pd.read_pickle(os.path.join(tap_file_dir, 'tap_shape.pickle'))

In [25]:
tap_shapes_df.columns

Index(['stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'zone_id', 'stop_url',
       'location_type', 'parent_station', 'stop_timezone',
       'wheelchair_boarding', 'stop_code', 'platform_code', 'position',
       'direction', 'stop_id', 'osm_node_id', 'shst_node_id', 'model_node_id',
       'X', 'Y', 'tap_id', 'tap_X', 'tap_Y', 'id', 'shstGeometryId',
       'geometry'],
      dtype='object')

In [5]:
node_id_crosswalk_df = pd.read_csv(node_id_crosswalk_file)

# Update ID Process

In [9]:
def update_stop_node_id(x, node_id_dict):
    if x in list(node_id_dict.keys()):
        return node_id_dict[x]
    else:
        return x

In [10]:
node_crosswalk_dict = dict(zip(node_id_crosswalk_df.model_node_id_old, node_id_crosswalk_df.model_node_id_new))

In [13]:
new_tap_links_df = tap_links_df.copy()

new_tap_links_df['A'] = new_tap_links_df['A'].apply(lambda x: update_stop_node_id(x, node_crosswalk_dict))
new_tap_links_df['B'] = new_tap_links_df['B'].apply(lambda x: update_stop_node_id(x, node_crosswalk_dict))

In [22]:
new_tap_links_df[new_tap_links_df.A == 1027677][["stop_name","A", "B"]]

,stop_name,A,B
4653,Taylor St & Bay St,1027677,90098


In [20]:
tap_links_df[tap_links_df.A == 1027676][["stop_name","A", "B"]]

,stop_name,A,B
4653,Taylor St & Bay St,1027676,90098


In [26]:
new_tap_shapes_df = tap_shapes_df.copy()

new_tap_shapes_df['model_node_id'] = new_tap_shapes_df['model_node_id'].apply(
    lambda x: update_stop_node_id(x, node_crosswalk_dict))

In [27]:
tap_shapes_df[tap_shapes_df.model_node_id == 1027676]

,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,...,shst_node_id,model_node_id,X,Y,tap_id,tap_X,tap_Y,id,shstGeometryId,geometry
5888,Taylor St & Bay St,NaN,37.805257,-122.415307,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1027676,-122.415307,37.805257,1229,-122.415201,37.80418,tap_4654,tap_4654,"LINESTRING (-122.41531 37.80526, -122.41520 37..."


In [29]:
new_tap_shapes_df[new_tap_shapes_df.model_node_id == 1027677]

,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,...,shst_node_id,model_node_id,X,Y,tap_id,tap_X,tap_Y,id,shstGeometryId,geometry
5888,Taylor St & Bay St,NaN,37.805257,-122.415307,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1027677,-122.415307,37.805257,1229,-122.415201,37.80418,tap_4654,tap_4654,"LINESTRING (-122.41531 37.80526, -122.41520 37..."


# write out update files

In [32]:
new_tap_links_df.to_pickle(os.path.join(out_dir, 'tap_link.pickle'))
new_tap_shapes_df.to_pickle(os.path.join(out_dir, 'tap_shape.pickle'))